## Imports

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sea
import geodatasets as gds
import fiona #geopandas needs this for shapefiles
from geopy.distance import geodesic
from matplotlib import colormaps
from shapely.geometry import Point

pd.set_option('display.max_columns', 70) # Set max display to 70 columns
pd.set_option('display.float_format', '{:.0f}'.format) #Causes pandas do NOT output integers as exponents

## Importing SDOH et al data, as well as geospatial data (census tract, municipality, and law enforcement boundaries) to join on tract, county or municipality FIPS:

In [ ]:
ahrq_df = pd.read_csv(r"/Users/kristinknippenberg/My Drive/0_BIOINFORMATICS/BMI 6016 Data Wrangling/Group 4 Project/Data Files/SDOH_2020_tract_Utah (reduce to variables).csv")
#AHRQ SDOH variables for Gini Index, Poverty, Median houshold income, Unemployment, Uninsured, Citizenship, Foreign-born, UT population above 1-year-old
air_quality_df = pd.read_csv(r"/Users/kristinknippenberg/My Drive/0_BIOINFORMATICS/BMI 6016 Data Wrangling/Group 4 Project/Data Files/annual_conc_by_monitor_2020.csv") # Air Quality
asthma_df = pd.read_csv(r"/Users/kristinknippenberg/My Drive/0_BIOINFORMATICS/BMI 6016 Data Wrangling/Group 4 Project/Data Files/PLACES_Utah_ct_Asthma.csv") #create Asthma data df on CASTHMA_CrudePrev
crime_df = pd.read_csv(r"/Users/kristinknippenberg/My Drive/0_BIOINFORMATICS/BMI 6016 Data Wrangling/Group 4 Project/Data Files/2022crime.csv") #Crime on "CRIMES_PER_1K"
resp_haz_df = pd.read_csv(r"/Users/kristinknippenberg/My Drive/0_BIOINFORMATICS/BMI 6016 Data Wrangling/Group 4 Project/Data Files/CancerRisk_by_block_pollutants.csv") #create Respiratory Hazard data df (group by tract) on "Total Cancer Risk (per million) "
svi_df = pd.read_csv(r"/Users/kristinknippenberg/My Drive/0_BIOINFORMATICS/BMI 6016 Data Wrangling/Group 4 Project/Data Files/Utah2020SVI (reduce RPL_THEMES).csv") #Social Vulnerability Index on "RPL_THEMES"
urban_rural_df = pd.read_csv(r"/Users/kristinknippenberg/My Drive/0_BIOINFORMATICS/BMI 6016 Data Wrangling/Group 4 Project/Data Files/Utah_UAs_Block (group).csv") #create Urban-Rural data df (group by tract)
walkable_df = pd.read_csv(r"/Users/kristinknippenberg/My Drive/0_BIOINFORMATICS/BMI 6016 Data Wrangling/Group 4 Project/Data Files/Utah_Walkability (reduce).csv") #Walkability

#CENSUS TRACT BASEMAP (which has GEOID20 as well as county FIPS)
utct_base = r"/Users/kristinknippenberg/My Drive/0_BIOINFORMATICS/BMI 6016 Data Wrangling/Group 4 Project/GeoFiles/CensusTracts2020.shp"
utct_base_df = gpd.read_file(utct_base)

#MUNICIPALITY BOUNDARIES (which has municipality FIPS)
ut_municp = r"/Users/kristinknippenberg/My Drive/0_BIOINFORMATICS/BMI 6016 Data Wrangling/Group 4 Project/GeoFiles/Municipalities.shp"
ut_municp_df = gpd.read_file(ut_municp)

#LAW ENFORCEMENT BOUNDARIES (which has just NAME)
utle_base = r"/Users/kristinknippenberg/My Drive/0_BIOINFORMATICS/BMI 6016 Data Wrangling/Group 4 Project/GeoFiles/LawEnforcementBoundaries.shp"
utle_base_df = gpd.read_file(utle_base)

## Steps

In [ ]:
#Put em all in lists: 
all_dfs = [ahrq_df, air_quality_df, asthma_df, crime_df, resp_haz_df, svi_df, urban_rural_df, ut_municp_df, utct_base_df, utle_base_df, walkable_df]
all_df_names = ['ahrq_df', 'air_quality_df', 'asthma_df', 'crime_df', 'resp_haz_df', 'svi_df', 'urban_rural_df', 'ut_municp_df', 'utct_base_df', 'utle_base_df', 'walkable_df']
data_dfs = [ahrq_df, air_quality_df, asthma_df, crime_df, resp_haz_df, svi_df, urban_rural_df, walkable_df]
data_df_names = ['ahrq_df', 'air_quality_df', 'asthma_df', 'crime_df', 'resp_haz_df', 'svi_df', 'urban_rural_df', 'walkable_df']

### Convert 'objects' to 'strings' before further manipulation

In [ ]:
#First we'll get some info
for df in all_dfs:
    df.info()

#The most common reason for an 'object' dtype is that the column holds different types of data. 
#Pandas is known for using 'object' as a catch-all for columns that don't fit into a specific numeric or string type. 
#However, a quirk of Pandas is ALSO to label "string" datatypes as "objects." 
#First, We're going to check the dtypes in the object columns.
    obj_cols = df.select_dtypes(include='object').columns
    for n in obj_cols:
        if n in df.columns:
            print(f"{df[n].apply(type).value_counts()}\n")
            
#Now, click anywhere in the output cell, hit 'Ctrl-F', and search for '<class'. You can quickly see if these are all 'str' or can be changed to 'str'.

In [ ]:
#And IF all are strings (spoiler: they are), run code to officially convert all "object" datatypes to "string" datatypes:
for df in all_dfs:
    
    obj_cols = df.select_dtypes(include='object').columns  
    for n in obj_cols:
        if n in df.columns:
            df[n] = df[n].astype("string")

### We're going to delete extraneous data from the data dfs, create some new composite variables, and adjust all the joining variables, making sure these are strings of 11 digits with a label that matches the basemap(s).

#### AHRQ

In [ ]:
#Slim down the df
ahrq_kept_var = ['YEAR', 'TRACTFIPS', 'COUNTYFIPS', 'STATEFIPS', 'STATE', 'COUNTY', 'REGION', 'TERRITORY', 'ACS_TOT_POP_US_ABOVE1', 'ACS_TOT_POP_ABOVE25', 'ACS_PCT_CTZ_US_BORN', 'ACS_PCT_FOREIGN_BORN', 'ACS_PCT_WHITE', 'ACS_PCT_HISPANIC', 'ACS_MEDIAN_HH_INC', 'ACS_PCT_OWNER_HU', 'ACS_PCT_OWNER_HU_COST_30PCT', 'ACS_PCT_RENTER_HU', 'ACS_PCT_RENTER_HU_COST_30PCT', 'ACS_PCT_VACANT_HU', 'ACS_PCT_UNEMPLOY', 'ACS_PCT_UNINSURED', 'ACS_PCT_PERSON_INC_BELOW99', 'ACS_GINI_INDEX', 'ACS_PCT_LT_HS', 'ACS_PCT_HS_GRADUATE', 'ACS_PCT_COLLEGE_ASSOCIATE_DGR', 'ACS_PCT_BACHELOR_DGR', 'ACS_PCT_GRADUATE_DGR']
ahrq_dff = ahrq_df[ahrq_kept_var].copy()

#Create new variables from old
ahrq_dff['ACS_PCT_NONHISP'] = 100-(ahrq_dff['ACS_PCT_HISPANIC']) #Getting % Non-hispanic

ahrq_dff['PCT_EDUCATION_WT_AVG'] = (((ahrq_dff['ACS_PCT_LT_HS']*ahrq_dff['ACS_TOT_POP_ABOVE25']*1)+(ahrq_dff['ACS_PCT_HS_GRADUATE']*ahrq_dff['ACS_TOT_POP_ABOVE25']*2)+(ahrq_dff['ACS_PCT_COLLEGE_ASSOCIATE_DGR']*ahrq_dff['ACS_TOT_POP_ABOVE25']*3)+(ahrq_dff['ACS_PCT_BACHELOR_DGR']*ahrq_dff['ACS_TOT_POP_ABOVE25']*4)+(ahrq_dff['ACS_PCT_GRADUATE_DGR']*ahrq_dff['ACS_TOT_POP_ABOVE25']*5))/ahrq_dff['ACS_TOT_POP_ABOVE25'])/100
ahrq_dff['PCT_EDUCATION_WT_AVG'] = ahrq_dff['PCT_EDUCATION_WT_AVG'].round(2) #Getting a weighted average of educational attainment, less than HS through graduate degree, for people over age 25

#Fix the joining variable and put it near the other variables (it has 11 digits already; just needs to be a string and renamed)
ahrq_dff['GEOID20'] = ahrq_dff['TRACTFIPS'].astype('string')
ahrq_dff = ahrq_dff[['YEAR', 'GEOID20', 'COUNTYFIPS', 'STATEFIPS', 'STATE', 'COUNTY', ] + [col for col in ahrq_dff.columns if col not in ['YEAR', 'GEOID20', 'COUNTYFIPS', 'STATEFIPS', 'STATE', 'COUNTY', ]]]

#ahrq_dff.head()
#ahrq_dff.info()

#### Air Quality

In [ ]:
# Filtering the AQ Data
air_quality_df = air_quality_df[air_quality_df['State Code'] == 49] # Utah
air_quality_df = air_quality_df[air_quality_df['Parameter Name'] == 'PM2.5 - Local Conditions'] # PM 2.5
air_quality_df = air_quality_df[air_quality_df['Pollutant Standard'].isin(['PM25 24-hour 2024', 'PM25 Annual 2024'])] # PM 2.5 standard effective 2024

In [ ]:
air_quality_df.head()

In [ ]:
utct_base_df.head()

In [ ]:
print(utct_base_df.crs)
# Convert the utct_base_df GeoDF from EPSG:3857 (NAD83) to EPSG:4326 (WGS 84)
# This will match the format of the aq_geo_df
utct_base_df = utct_base_df.to_crs(epsg=4326)
print(utct_base_df.crs)

In [ ]:
# Creating point geometries from latitude and longitude

geometry = [Point(xy) for xy in zip(air_quality_df['Longitude'], air_quality_df['Latitude'])]
aq_geo_df = gpd.GeoDataFrame(air_quality_df, geometry=geometry, crs="EPSG:4326")  # The air quality data is in WGS 84 (EPSG:4326)

In [ ]:
# Spatial join between the point GeoDF and census tract DF
joined_df = gpd.sjoin(aq_geo_df, utct_base_df, predicate='within', how='left') # Using left join to keep all original data.

In [ ]:
# Extract GEOID from joined GeoDF
if 'GEOID' in joined_df.columns:
    result_df = joined_df[['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC', 'Latitude', 'Longitude', 'GEOID']]
elif 'GEOID20' in joined_df.columns:
    result_df = joined_df[['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC', 'Latitude', 'Longitude', 'GEOID20']]
else:
    result_df = joined_df[['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC', 'Latitude', 'Longitude']] # if no GEOID information, keep original data.
    print("Warning: No GEOID or GEOID20 column found in census tract data.")

In [ ]:
# Print head of joined_df with unique TRACTCE20 values (unique census tracts)
joined_df.loc[joined_df.drop_duplicates(subset='TRACTCE20').index].head()
joined_df.info()

In [ ]:
print(utct_base_df.crs, joined_df.crs)
# Convert the utct_base_df GeoDF from EPSG:4326 (WGS 84) back to EPSG:3857 (NAD83), for the sake of the rest of the data and plots
# This will match the format of the aq_geo_df
utct_base_df = utct_base_df.to_crs(epsg=3857)
air_quality_dff = joined_df.to_crs(epsg=3857)
print(utct_base_df.crs, air_quality_dff.crs)

#### Asthma

In [ ]:
#Calculate % of population: 
asthma_df['CASTHMA_PCT'] = (asthma_df['CASTHMA_CrudePrev']/asthma_df['TotalPopulation']) * 100
asthma_df['CASTHMA_PCT'] = asthma_df['CASTHMA_PCT'].round(3)

#Fix the joining variable (it has 11 digits already; just needs to be a string and renamed) and put it near the other variables
asthma_df['GEOID20'] = asthma_df['TractFIPS'].astype('string')
asthma_df = asthma_df[['StateAbbr', 'StateDesc', 'CountyName', 'CountyFIPS', 'GEOID20'] + [col for col in asthma_df.columns if col not in ['StateAbbr', 'StateDesc', 'CountyName', 'CountyFIPS', 'GEOID20']]]

#asthma_df.head()
#asthma_df.info()

#### Crime

In [ ]:
#Transform the crime data from counts into 'per 1000':
crime_per1K_df = crime_df.copy()

for c in crime_per1K_df.columns:
    if c not in ['NAME', 'COUNTYFP20', 'FIPS', 'Population']:
        crime_per1K_df[c] = (crime_df[c] / crime_df['Population']) *1000

#Fix the joining variables (just need to be strings)
crime_per1K_df['COUNTYFP20'] = crime_per1K_df['COUNTYFP20'].astype('string')
crime_per1K_df['FIPS'] = crime_per1K_df['FIPS'].astype('string')
crime_per1K_df['TOT_CRIME_P1K'] = crime_per1K_df['Total']

#crime_per1K_df.head()
crime_per1K_df.info()

#### Respiratory Hazard

In [ ]:
#Given the nature of AIR DATA, I think I'm going to return the maximum value ...
resp_haz_max_df = resp_haz_df.groupby('FIPS')['Total Cancer Risk (per million)'].max().reset_index()

# ... but it turns out the "blocks" roll all the way up to counties (n = 29), not census tracts, as promised. They did a short-cut in 2020. Bleh.
# So we're not going to be able to join on the census tracts; we need to join on the counties, which in the census tract basemap, are 3 digit codes.
# We will fix the joining variable to be a string and renamed, and we will also slice the values from 5 digits down to 3 by removing the "49" prefix.
resp_haz_max_df['COUNTYFP20'] = resp_haz_max_df['FIPS'].astype("string")
resp_haz_max_df['COUNTYFP20'] = resp_haz_max_df['COUNTYFP20'].str[2:]

#And we're going to shorten our data variable:
resp_haz_max_df['TCRPM'] = resp_haz_max_df['Total Cancer Risk (per million)'].copy()

#Finally we will order the variables
resp_haz_max_df = resp_haz_max_df[['FIPS', 'COUNTYFP20', 'Total Cancer Risk (per million)'] + [col for col in resp_haz_max_df.columns if col not in ['FIPS', 'COUNTYFP20', 'Total Cancer Risk (per million)']]]

#resp_haz_max_df.head()
#resp_haz_max_df.info()

#### SVI: overall summary ranking and limited English

In [ ]:
# Slim down the df
svi_kept_var = ['ST', 'STATE', 'ST_ABBR', 'STCNTY', 'COUNTY', 'FIPS', 'LOCATION', 'AREA_SQMI', 'E_TOTPOP', 'RPL_THEMES', 'EP_LIMENG']
svi_dff = svi_df[svi_kept_var].copy()

#Fix the joining variable (it has 11 digits already; just needs to be a string and renamed) and put it near the other variables
svi_dff['GEOID20'] = svi_dff['FIPS'].astype("string")
svi_dff = svi_dff[['ST', 'STATE', 'ST_ABBR', 'STCNTY', 'COUNTY', 'GEOID20'] + [col for col in svi_dff.columns if col not in ['ST', 'STATE', 'ST_ABBR', 'STCNTY', 'COUNTY', 'GEOID20']]]

#svi_dff.head()
#svi_dff.info()

#### Urban Areas (only for census-defined "Urban Areas." Any tracts without data are assumed "rural.")

In [ ]:
#Turn the joining variables into strings
urban_rural_df['STATE'] = urban_rural_df['STATE'].astype('string')
urban_rural_df['COUNTY'] = urban_rural_df['COUNTY'].astype('string')
urban_rural_df['TRACT'] = urban_rural_df['TRACT'].astype('string')
urban_rural_df['UA_POP'] = urban_rural_df['2020_POP'].rename()

#Concatenate the above variables into an 11-digit code of the proper name "GEOID20"
urban_rural_df['GEOID20'] = urban_rural_df['STATE'] + urban_rural_df['COUNTY'].apply(lambda x: str(x).zfill(3)) + urban_rural_df['TRACT'].astype('string')

#And put the variables in a preferred order
urban_rural_df = urban_rural_df[['STATE', 'COUNTY', 'TRACT', 'GEOID20', 'UA_POP'] + [col for col in urban_rural_df.columns if col not in ['STATE', 'COUNTY', 'TRACT', 'GEOID20', 'UA_POP']]]
urban_rural_dff = urban_rural_df.groupby('GEOID20')['UA_POP'].sum().reset_index()

#urban_rural_dff.head()
#urban_rural_dff.info()

#### Walkability, rolling up to census tract

In [ ]:
#Turn the joining variables into strings
walkable_df['STATEFP'] = walkable_df['STATEFP'].astype('string')
walkable_df['COUNTYFP'] = walkable_df['COUNTYFP'].astype('string')
walkable_df['TRACTCE'] = walkable_df['TRACTCE'].astype('string')

#Concatenate the above variables into an 11-digit code of the proper name "GEOID20"
walkable_df['GEOID20'] = walkable_df['STATEFP'] + walkable_df['COUNTYFP'].apply(lambda x: str(x).zfill(3)) + walkable_df['TRACTCE'].apply(lambda x: str(x).zfill(6)).astype('string')

#And calculate the mean walkability (rounded to 2 decimals) for each GEOID20
walkable_dff = walkable_df.groupby('GEOID20')['NatWalkInd'].mean().reset_index()
walkable_dff['NAT_WALK_IND'] = walkable_dff['NatWalkInd'].round(2)

#walkable_dff.head()
#walkable_dff.info()

### Preparing to Join and Merge

In [ ]:
#REVISE the lists! 
all_dfs2 = [ahrq_dff, air_quality_dff, asthma_df, crime_per1K_df, resp_haz_max_df, svi_dff, urban_rural_dff, ut_municp_df, utct_base_df, utle_base_df, walkable_dff]
all_df_names2 = ['ahrq_dff', 'air_quality_dff', 'asthma_df', 'crime_per1K_df', 'resp_haz_max_df', 'svi_dff', 'urban_rural_dff', 'ut_municp_df', 'utct_base_df', 'utle_base_df', 'walkable_dff']
data_dfs2 = [ahrq_dff, air_quality_dff, asthma_df, crime_per1K_df, resp_haz_max_df, svi_dff, urban_rural_dff, walkable_dff]
data_df_names2 = ['ahrq_dff', 'air_quality_dff', 'asthma_df', 'crime_per1K_df', 'resp_haz_max_df', 'svi_dff', 'urban_rural_dff', 'walkable_dff']

### Double-click on me:

We've got the following dfs with the following "JOINING" variable, the base it joins to, and "DATA" variable:

DATAFRAME           JOINING_VAR             BASEMAP         DATA_VAR                METRIC
ahrq_dff            GEOID20                 utct_base_df    *                       %, MEDIAN, INDEX, WT_AVG
air_quality_dff     GEOID20                 utct_base_df    POC                     ????
asthma_df           GEOID20                 utct_base_df    CASTHMA_PCT             %
crime_per1K_df      NAME,COUNTYFP20,FIPS    all 3           TOT_CRIME_P1K       per 1000
resp_haz_max_df     COUNTYFP20              utct_base_df    TCRPM                   Max
svi_dff             GEOID20                 utct_base_df    RPL_THEMES,EP_LIMENG    PERCENTILE RANKING,%
urban_rural_dff     GEOID20                 utct_base_df    UA_POP                  COUNT
walkable_dff        GEOID20                 utct_base_df    NAT_WALK_IND            INDEX

*'ACS_PCT_CTZ_US_BORN', 'ACS_PCT_FOREIGN_BORN', 'ACS_PCT_WHITE', 'ACS_PCT_NONHISP', 'ACS_MEDIAN_HH_INC', 'ACS_PCT_OWNER_HU', 'ACS_PCT_OWNER_HU_COST_30PCT', 'ACS_PCT_RENTER_HU', 'ACS_PCT_RENTER_HU_COST_30PCT', 'ACS_PCT_VACANT_HU', 'ACS_PCT_UNEMPLOY', 'ACS_PCT_UNINSURED', 'ACS_PCT_PERSON_INC_BELOW99', 'ACS_GINI_INDEX', 'PCT_EDUCATION_WT_AVG'

### Separate CSVs for combining in SQL or similar:

In [ ]:
for df, name in zip(all_dfs2, all_df_names2):
    df.to_csv(f"{name}.csv", index=False)
    
#One of us should set this up in SQL and query!

### Attempting to make one flat CSV:

In [ ]:
#One CSV

one = pd.merge(ahrq_dff[['GEOID20','ACS_PCT_CTZ_US_BORN', 'ACS_PCT_FOREIGN_BORN', 'ACS_PCT_WHITE', 'ACS_PCT_NONHISP', 'ACS_MEDIAN_HH_INC', 'ACS_PCT_OWNER_HU', 'ACS_PCT_OWNER_HU_COST_30PCT', 'ACS_PCT_RENTER_HU', 'ACS_PCT_RENTER_HU_COST_30PCT', 'ACS_PCT_VACANT_HU', 'ACS_PCT_UNEMPLOY', 'ACS_PCT_UNINSURED', 'ACS_PCT_PERSON_INC_BELOW99', 'ACS_GINI_INDEX', 'PCT_EDUCATION_WT_AVG']], asthma_df[['GEOID20', 'CASTHMA_PCT']], on='GEOID20', how='left')

one_df = pd.merge(one, svi_dff[['GEOID20', 'RPL_THEMES', 'EP_LIMENG']], on='GEOID20', how='left')

one_df_to = pd.merge(one_df, urban_rural_dff[['GEOID20', 'UA_POP']], on='GEOID20', how='left')

one_df_to_rule = pd.merge(one_df_to, walkable_dff[['GEOID20', 'NAT_WALK_IND']], on='GEOID20', how='left')

one_df_to_rule_them = pd.merge(one_df_to_rule, utct_base_df,on='GEOID20', how='left')

one_df_to_rule_them_all = pd.merge(one_df_to_rule_them, resp_haz_max_df, on='COUNTYFP20', how='left')

#one_df_to_rule_them_all.info()
#one_df_to_rule_them_all.head()
#one_df_to_rule_them_all.to_csv('one_df_to_rule_them_all.csv', index=False)

#At this point, re the CSV file, Excel is telling me "This data set is too large for the Excel grid.  If you save this workbook, you'll lose data that wasn't loaded". I think it's all the geospatial data that's gumming it up.
# And I still have to merge the crime data with the LE boundaries and municipalities.

### Now we're going to try to develop an easy interface where we can just add/remove different layers and have them show up on the map.
#### Plot base layers (census tract and law enforcement)

In [ ]:
# Plot base layers (census tract and law enforcement) and additional data layers dynamically
def ut_base_df(*layers):
    ax = utct_base_df.boundary.plot(
        figsize=(9,6.5),
        linewidth=0.25,
        edgecolor="000000",
        facecolor='none'
        
    )
    utle_base_df.plot( # Plot the law enforcement map onto Census Tract Map, but make it invisible
        ax=ax,  # Plot this dataframe on top of the boundary plot
        linewidth=0.25,
        edgecolor="none",
        facecolor="none"
    )
    
    ax.set_axis_off()  
    return ax  # Return the axis to plot further layers on top

plt.show(ut_base_df())

#### Prepare Crime layer

In [ ]:
crime_lay = pd.merge(utle_base_df, crime_per1K_df, on='NAME')
crime_layer = crime_lay.plot(
    column='TOT_CRIME_P1K',
    cmap='coolwarm',
    alpha=0.6,
    legend=True,
    edgecolor="000000",
    linewidth=0.25,
    figsize=(9, 6.5)
)

#### Prepare the respiratory hazard layer

In [ ]:
resp_haz_lay = pd.merge(utct_base_df, resp_haz_max_df, on='COUNTYFP20')
resp_haz_layer = resp_haz_lay.plot(
    column='TCRPM',
    cmap='coolwarm',
    alpha=0.6,
    legend=True,
    edgecolor="000000",
    linewidth=0.25,
    figsize=(9, 6.5)
)

#### Prepare the rest of the layers

In [ ]:
layers = ['ACS_GINI_INDEX', 'ACS_MEDIAN_HH_INC', 'ACS_PCT_CTZ_US_BORN', 'ACS_PCT_FOREIGN_BORN', 'ACS_PCT_NONHISP', 'ACS_PCT_OWNER_HU', 'ACS_PCT_OWNER_HU_COST_30PCT', 'ACS_PCT_PERSON_INC_BELOW99', 'ACS_PCT_RENTER_HU', 'ACS_PCT_RENTER_HU_COST_30PCT', 'ACS_PCT_UNEMPLOY', 'ACS_PCT_UNINSURED', 'ACS_PCT_VACANT_HU', 'ACS_PCT_WHITE', 'CASTHMA_PCT', 'EP_LIMENG', 'NAT_WALK_IND', 'PCT_EDUCATION_WT_AVG', 'POC', 'RPL_THEMES', 'UA_POP']

df_source = [ahrq_dff, ahrq_dff, ahrq_dff, ahrq_dff, ahrq_dff, ahrq_dff, ahrq_dff, ahrq_dff, ahrq_dff, ahrq_dff, ahrq_dff, ahrq_dff, ahrq_dff, ahrq_dff, asthma_df, svi_dff, walkable_dff, ahrq_dff, air_quality_dff, svi_dff, urban_rural_dff]

layers_dict = {}

In [ ]:
for l, source in zip(layers, df_source):
    # Merge dataframes
    lay_df = pd.merge(utct_base_df, source[['GEOID20', l]], on='GEOID20', how='left')
    
    # Plot the data and store the plot object in the dictionary
    layer_plot = lay_df.plot(
        column=l,
        cmap='coolwarm',
        alpha=0.6,
        legend=True,
        edgecolor="000000",
        linewidth=0.25,
        figsize=(9, 6.5)
    )
    
    # Save the plot object in the dictionary with the layer name as the key
    layers_dict[l] = layer_plot

# Now layers_dict contains all the plots, and you can access them like layers_dict['ACS_GINI_INDEX'] to view the plot.

#### Adjust the color-maps

In [ ]:
# Access the plot for the 'ACS_GINI_INDEX' layer
layer_plot = layers_dict['ACS_GINI_INDEX']  # This is the returned Axes object

# Access the collection (which is a list of the plotted shapes or patches)
collection = layer_plot.collections[0]  # The first collection is the actual plot

# Change the colormap to 'coolwarm'
collection.set_cmap('coolwarm')

# If needed, also adjust the color limits
#collection.set_clim(0, 100)  # Adjust color limits (you can change these values as appropriate)

# Optionally, update the colorbar if it was already added
layer_plot.figure.colorbar(collection)

# Redraw the plot
#plt.show()

print(layer_plot)